In [2]:
import pandas as pd
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.max_rows', 500)
import pickle
from jupyter_dash import JupyterDash
import dash
from dash import dcc 
from dash import html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc

import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize)

import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance, ImageDraw,ImageFont
import seaborn as sns
#sns.set()
import arviz as az
import pymc3 as pm
print(pm.__version__)
import theano.tensor as tt
import patsy

import os
import re
import glob
import random
# import plotnine
from sklearn import preprocessing
from tqdm import tqdm

# import plotnine 
# from plotnine.data import economics 
# from plotnine import * 
import plotly.express as px

from skimage import measure, restoration,morphology
from skimage import io, filters, measure, color, img_as_ubyte
from skimage.draw import disk
from skimage import measure, restoration,morphology

RANDOM_SEED = 8927
np.random.seed(RANDOM_SEED)


%config InlineBackend.figure_format = 'retina'
os.chdir(r'F:\HAB_2\PrinzScreen\training_classfication')
from utils import AIPS_cellpose as AC
from utils import AIPS_file_display as AFD

3.11.5


Preforming multivariate logistic regression classification  
Save six mixed images based on manual classification:

input_class: 'F:\HAB_2\PrinzScreen\output_lab_meeting\kerenl_60_mix\input_mix'\
input_image : 'F:\HAB_2\PrinzScreen\output_lab_meeting\kerenl_60_mix\input_mix'\
output: 'F:\HAB_2\PrinzScreen\output_lab_meeting\kerenl_60_mix' 


In [3]:
path =  r'F:\HAB_2\PrinzScreen\output_lab_meeting\kerenl_60_mix\input_mix'
os.chdir(path)
images_name = glob.glob("*.tif")
AIPS_pose_object = AC.AIPS_cellpose(Image_name = images_name[0], path= path, model_type="cyto", channels=[0,0])
img = AIPS_pose_object.cellpose_image_load()
# create mask for the entire image
mask, table = AIPS_pose_object.cellpose_segmantation(image_input=img[0,:,:])
table_gran_temp = AIPS_pose_object.measure_properties(input_image=img[0,:,:])

In [4]:
table_gran_temp.keys()

dict_keys(['label', 'centroid-0', 'centroid-1', 'area', 'eccentricity', 'euler_number', 'extent', 'feret_diameter_max', 'inertia_tensor-0-0', 'inertia_tensor-0-1', 'inertia_tensor-1-0', 'inertia_tensor-1-1', 'inertia_tensor_eigvals-0', 'inertia_tensor_eigvals-1', 'orientation', 'perimeter', 'perimeter_crofton', 'solidity', 'sd_intensity', 'skew_intensity', 'pixelcount', 'mean_int'])

In [23]:
def pm_table_from_image(path,image_number = 1,kernel_lop = False,kernel_size=20):
    os.chdir(path)
    images_name = glob.glob("*.tif")
    AIPS_pose_object = AC.AIPS_cellpose(Image_name = images_name[image_number], path= path, model_type="cyto", channels=[0,0])
    img = AIPS_pose_object.cellpose_image_load()
    # create mask for the entire image
    mask, table = AIPS_pose_object.cellpose_segmantation(image_input=img[0,:,:])
    table_gran_temp = AIPS_pose_object.measure_properties(input_image=img[0,:,:])
    table_gran = pd.DataFrame(table_gran_temp['mean_int'].tolist())
    # kernel loop:
    if kernel_lop:
        open_vec = np.linspace(1, 80, kernel_size, endpoint=False, dtype=int)
        for i in range(1, len(open_vec)):
            selem = morphology.disk(open_vec[i], dtype=bool)
            eros_pix = morphology.erosion(img[0,:,:], selem=selem)
            rec = morphology.dilation(eros_pix, selem=selem)
            table_gran_temp = AIPS_pose_object.measure_properties(input_image=rec)
            table_gran[int(open_vec[i])] = table_gran_temp['mean_int'].tolist()
        table_ = pd.DataFrame({'label':table.index.values,'area':table['area'].values,'centroid-0':table['centroid-0'].values,'centroid-1':table['centroid-1'].values})
        table_gran_comp = pd.concat((table_,table_gran),1)
        table_gran_comp = table_gran_comp.melt(id_vars=["label", "area","centroid-0","centroid-1"])
        table_out = table_gran_comp.sort_values(['label','variable']).reset_index(drop=True)
        table_out = table_out.rename(columns={"label":"image_group","variable": "raius_list", "value": "image_signal"})
        from sklearn import preprocessing
        table_sel = table_out
        # class_name =  table_sel['class'].values
        image_group =  table_sel['image_group'].values
        le = preprocessing.LabelEncoder()
        # encoding
        # class_name_encoded=le.fit_transform(class_name).tolist()
        image_group_encoded=le.fit_transform(image_group).tolist()
        table_dict={'image_signal':table_sel.image_signal.values.tolist(),
                    'raius_list':table_sel.raius_list.values.tolist(),
                    'image_group':image_group_encoded}
        table_fin = pd.DataFrame(table_dict)
        table_fin['image_group'] = pd.Categorical(table_fin['image_group'], ordered=False)
        return table_fin, mask, img,table_
    else:
        selem = morphology.disk(kernel_size, dtype=bool)
        eros_pix = morphology.erosion(img[0,:,:], selem=selem)
        rec = morphology.dilation(eros_pix, selem=selem)
        table_gran_temp = AIPS_pose_object.measure_properties(input_image=rec)
        table_gran[int(kernel_size)] = table_gran_temp['mean_int'].tolist()
        table_gran_temp = pd.DataFrame(table_gran_temp)
        table_ = pd.DataFrame({'label':table_gran_temp.index.values,'area':table_gran_temp['area'].values,'centroid-0':table_gran_temp['centroid-0'].values,
                       'centroid-1':table_gran_temp['centroid-1'].values, 'sd_intensity':table_gran_temp['sd_intensity'].values , 
                       'skew_intensity':table_gran_temp['skew_intensity'].values, 'pixelcount':table_gran_temp['pixelcount'].values,
                       'mean_int':table_gran_temp['mean_int'].values})
        table_gran_comp = pd.concat((table_,table_gran),1)
        table_fin = table_gran_comp
        
        blank = np.zeros_like(img[0,:,:])
        img_gs = img_as_ubyte(blank)
        PIL_image = Image.fromarray(img_gs)
        # round
        info_table = table_gran_comp.round({'centroid-0': 0, 'centroid-1': 0})
        #     info_table['predict_round'] = info_table.loc[:, 'predict'].astype(float).round(round_n)
        #     info_table['area_round'] = info_table.loc[:, 'area'].astype(float).round(round_n)
        info_table = info_table.reset_index(drop=True)
        draw = ImageDraw.Draw(PIL_image)
        # use a bitmap font
        font = ImageFont.truetype("arial.ttf", 30, encoding="unic")
        for i in range(len(info_table)):
            draw.text((info_table.iloc[i, 3].astype('int64'), info_table.iloc[i, 2].astype('int64')),
                      str(info_table.iloc[i, 0]), 'red', font=font)
        PIL_image_input = Image.fromarray(img_as_ubyte(img[0,:,:]))
        return images_name[image_number],table_fin, mask, img,table_,PIL_image,PIL_image_input

In [24]:
path =  r'F:\HAB_2\PrinzScreen\output_lab_meeting\kerenl_60_mix\input_mix'
path_out = r'F:\HAB_2\PrinzScreen\output_lab_meeting\kerenl_60_mix'
for i in range(7):
    name_,table_fin, mask, img,table_,PIL_image,PIL_image_input = pm_table_from_image(path,image_number=i,
                                                                                      kernel_lop = False,kernel_size=60)
    table_fin.to_csv(os.path.join(path_out,'Table_mix_{}.csv'.format(name_)))
    np.save(os.path.join(path_out,'mask_{}.npy'.format(name_)),mask)
    np.save(os.path.join(path_out,'img_{}.npy'.format(name_)),img)
    plt.imsave(os.path.join(path_out,'img_lable_{}.png'.format(name_)),PIL_image)
    plt.imsave(os.path.join(path_out,'img_{}.png'.format(name_)),PIL_image_input)  


F:\Gil\anaconda\envs\pymc3_cellpose\lib\site-packages\ipykernel_launcher.py:40: FutureWarning: `selem` is a deprecated argument name for `erosion`. It will be removed in version 1.0. Please use `footprint` instead.
F:\Gil\anaconda\envs\pymc3_cellpose\lib\site-packages\ipykernel_launcher.py:41: FutureWarning: `selem` is a deprecated argument name for `dilation`. It will be removed in version 1.0. Please use `footprint` instead.
F:\Gil\anaconda\envs\pymc3_cellpose\lib\site-packages\ipykernel_launcher.py:49: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
F:\Gil\anaconda\envs\pymc3_cellpose\lib\site-packages\skimage\util\dtype.py:541: UserWarning: Downcasting uint16 to uint8 without scaling because max value 0 fits in uint8
  return _convert(image, np.uint8, force_copy)


In [ ]:
dict_keys(['label', 'centroid-0', 'centroid-1', 'area', 'eccentricity', 'euler_number', 'extent', 'feret_diameter_max', 'inertia_tensor-0-0', 'inertia_tensor-0-1', 'inertia_tensor-1-0', 'inertia_tensor-1-1', 'inertia_tensor_eigvals-0', 'inertia_tensor_eigvals-1', 'orientation', 'perimeter', 'perimeter_crofton', 'solidity', 'sd_intensity', 'skew_intensity', 'pixelcount', 'mean_int'])

In [ ]:
table_ = pd.DataFrame({'label':table_gran_temp.index.values,'area':table_gran_temp['area'].values,'centroid-0':table_gran_temp['centroid-0'].values,
                   'centroid-1':table_gran_temp['centroid-1'].values, 'sd_intensity':table_gran_temp['sd_intensity'].values , 
                   'skew_intensity':table_gran_temp['skew_intensity'].values, 'pixelcount':table_gran_temp['pixelcount'].values,
                   'mean_int':table_gran_temp['mean_int'].values,'eccentricity':table_gran_temp['eccentricity'].values,
                    'euler_number':table_gran_temp['euler_number'].values, 'extent':table_gran_temp['extent'].values,
                       'feret_diameter_max':table_gran_temp['feret_diameter_max'].values,'orientation':table_gran_temp['orientation'].values,
                      'perimeter':table_gran_temp['perimeter'].values,  'solidity':table_gran_temp['solidity'].values,  })

In [5]:
def pm_table_from_image(path,image_number = 1,kernel_lop = False,kernel_size=20):
    os.chdir(path)
    images_name = glob.glob("*.tif")
    AIPS_pose_object = AC.AIPS_cellpose(Image_name = images_name[image_number], path= path, model_type="cyto", channels=[0,0])
    img = AIPS_pose_object.cellpose_image_load()
    # create mask for the entire image
    mask, table = AIPS_pose_object.cellpose_segmantation(image_input=img[0,:,:])
    table_gran_temp = AIPS_pose_object.measure_properties(input_image=img[0,:,:])
    table_gran = pd.DataFrame(table_gran_temp['mean_int'].tolist())
    # kernel loop:
    if kernel_lop:
        open_vec = np.linspace(1, 80, kernel_size, endpoint=False, dtype=int)
        for i in range(1, len(open_vec)):
            selem = morphology.disk(open_vec[i], dtype=bool)
            eros_pix = morphology.erosion(img[0,:,:], selem=selem)
            rec = morphology.dilation(eros_pix, selem=selem)
            table_gran_temp = AIPS_pose_object.measure_properties(input_image=rec)
            table_gran[int(open_vec[i])] = table_gran_temp['mean_int'].tolist()
        table_ = pd.DataFrame({'label':table.index.values,'area':table['area'].values,'centroid-0':table['centroid-0'].values,'centroid-1':table['centroid-1'].values})
        table_gran_comp = pd.concat((table_,table_gran),1)
        table_gran_comp = table_gran_comp.melt(id_vars=["label", "area","centroid-0","centroid-1"])
        table_out = table_gran_comp.sort_values(['label','variable']).reset_index(drop=True)
        table_out = table_out.rename(columns={"label":"image_group","variable": "raius_list", "value": "image_signal"})
        from sklearn import preprocessing
        table_sel = table_out
        # class_name =  table_sel['class'].values
        image_group =  table_sel['image_group'].values
        le = preprocessing.LabelEncoder()
        # encoding
        # class_name_encoded=le.fit_transform(class_name).tolist()
        image_group_encoded=le.fit_transform(image_group).tolist()
        table_dict={'image_signal':table_sel.image_signal.values.tolist(),
                    'raius_list':table_sel.raius_list.values.tolist(),
                    'image_group':image_group_encoded}
        table_fin = pd.DataFrame(table_dict)
        table_fin['image_group'] = pd.Categorical(table_fin['image_group'], ordered=False)
        return table_fin, mask, img,table_
    else:
        selem = morphology.disk(kernel_size, dtype=bool)
        eros_pix = morphology.erosion(img[0,:,:], selem=selem)
        rec = morphology.dilation(eros_pix, selem=selem)
        table_gran_temp = AIPS_pose_object.measure_properties(input_image=rec)
        table_gran[int(kernel_size)] = table_gran_temp['mean_int'].tolist()
        table_gran_temp = pd.DataFrame(table_gran_temp)
        table_ = pd.DataFrame({'label':table_gran_temp.index.values,'area':table_gran_temp['area'].values,'centroid-0':table_gran_temp['centroid-0'].values,
                   'centroid-1':table_gran_temp['centroid-1'].values, 'sd_intensity':table_gran_temp['sd_intensity'].values , 
                   'skew_intensity':table_gran_temp['skew_intensity'].values, 'pixelcount':table_gran_temp['pixelcount'].values,
                   'mean_int':table_gran_temp['mean_int'].values,'eccentricity':table_gran_temp['eccentricity'].values,
                    'euler_number':table_gran_temp['euler_number'].values, 'extent':table_gran_temp['extent'].values,
                       'feret_diameter_max':table_gran_temp['feret_diameter_max'].values,'orientation':table_gran_temp['orientation'].values,
                      'perimeter':table_gran_temp['perimeter'].values,  'solidity':table_gran_temp['solidity'].values,  })
        table_gran_comp = pd.concat((table_,table_gran),1)
        table_fin = table_gran_comp
        
        blank = np.zeros_like(img[0,:,:])
        img_gs = img_as_ubyte(blank)
        PIL_image = Image.fromarray(img_gs)
        # round
        info_table = table_gran_comp.round({'centroid-0': 0, 'centroid-1': 0})
        #     info_table['predict_round'] = info_table.loc[:, 'predict'].astype(float).round(round_n)
        #     info_table['area_round'] = info_table.loc[:, 'area'].astype(float).round(round_n)
        info_table = info_table.reset_index(drop=True)
        draw = ImageDraw.Draw(PIL_image)
        # use a bitmap font
        font = ImageFont.truetype("arial.ttf", 30, encoding="unic")
        for i in range(len(info_table)):
            draw.text((info_table.iloc[i, 3].astype('int64'), info_table.iloc[i, 2].astype('int64')),
                      str(info_table.iloc[i, 0]), 'red', font=font)
        PIL_image_input = Image.fromarray(img_as_ubyte(img[0,:,:]))
        return images_name[image_number],table_fin, mask, img,table_,PIL_image,PIL_image_input

In [ ]:
path =  r'F:\HAB_2\PrinzScreen\output_lab_meeting\kerenl_60_mix\input_mix'
path_out = r'F:\HAB_2\PrinzScreen\output_lab_meeting\kerenl_60_mix\Properties'
for i in range(7):
    name_,table_fin, mask, img,table_,PIL_image,PIL_image_input = pm_table_from_image(path,image_number=i,
                                                                                      kernel_lop = False,kernel_size=60)
    table_fin.to_csv(os.path.join(path_out,'Table_mix_{}.csv'.format(name_)))
    np.save(os.path.join(path_out,'mask_{}.npy'.format(name_)),mask)
    np.save(os.path.join(path_out,'img_{}.npy'.format(name_)),img)
    plt.imsave(os.path.join(path_out,'img_lable_{}.png'.format(name_)),PIL_image)
    plt.imsave(os.path.join(path_out,'img_{}.png'.format(name_)),PIL_image_input)  

F:\Gil\anaconda\envs\pymc3_cellpose\lib\site-packages\ipykernel_launcher.py:40: FutureWarning: `selem` is a deprecated argument name for `erosion`. It will be removed in version 1.0. Please use `footprint` instead.
F:\Gil\anaconda\envs\pymc3_cellpose\lib\site-packages\ipykernel_launcher.py:41: FutureWarning: `selem` is a deprecated argument name for `dilation`. It will be removed in version 1.0. Please use `footprint` instead.
F:\Gil\anaconda\envs\pymc3_cellpose\lib\site-packages\ipykernel_launcher.py:52: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
F:\Gil\anaconda\envs\pymc3_cellpose\lib\site-packages\skimage\util\dtype.py:541: UserWarning: Downcasting uint16 to uint8 without scaling because max value 0 fits in uint8
  return _convert(image, np.uint8, force_copy)
